In [ ]:
!pip install fer

In [ ]:
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from cv2 import cv2
from fer import FER
import numpy as np

IMAGES_DIR = Path("/kaggle/input/img_align_celeba/img_align_celeba")
CVS_ATTR = Path("/kaggle/input/list_attr_celeba.csv")

df = pd.read_csv(CVS_ATTR)
detector = FER()

In [ ]:
def predict(img):
    prediction = detector.detect_emotions(img)

    if len(prediction) > 0:
        prediction = prediction[0]["emotions"]
        emotion_idx = np.argmax(list(prediction.values()))
        emotion = list(prediction.keys())[emotion_idx]
    else:
        emotion = "undetected"
    return emotion

In [ ]:
IMAGES = 6


# WIDGET ARCHITECTURE ----- ----- #
column_selector = widgets.Dropdown(
    options=df.columns,
    description="Select attribute",
    disabled=False)

toggle_attribute = widgets.ToggleButton(value=False, description="False",
                                   button_style="",
                                   icon="check")
toggle_sample = widgets.ToggleButton(value=False, description="Sample", button_style="")

bbox = HBox([column_selector, toggle_attribute, toggle_sample])


# EVENT HANDLER ----- ----- #
def eventhandler(column=None, show_neg=False, resample=None):
    att_filtr = -1 if show_neg else 1
    
    attr_mask = (df[column] == att_filtr)
    samples = df[attr_mask].sample(IMAGES)["image_id"]
    f = plt.figure(figsize=(7 * IMAGES, 8))
    
    for idx, image_id in enumerate(samples):
        ax = f.add_subplot(1, len(samples), idx+1)
        
        img = cv2.imread(str(IMAGES_DIR / image_id))
        emotion = predict(img)
        
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        ax.set_title(emotion, fontsize=40, y=1.08)
        ax.axis("off")
        plt.imshow(rgb_img)

    plt.show()
        
        
# WIDGET SHOW ----- ----- #
output = widgets.interactive(eventhandler, column=column_selector, 
                             show_neg=toggle_attribute, resample=toggle_sample).children[-1]
display(bbox)
display(output)